We'll use the yfinance library to fetch historical price data for a selected set of assets

In [13]:
import yfinance as yf
import pandas as pd

# Define the list of tickers
tickers = ['AAPL', 'MSFT', 'AMZN', 'JNJ', 'JPM', 'XOM', 'PG', 'GOOGL']

# Define the time period
start_date = '2018-01-01'
end_date = '2024-10-01'

# Fetch adjusted closing prices
prices = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Display the first few rows
print(prices.head())


[*********************100%***********************]  8 of 8 completed

Ticker                          AAPL       AMZN      GOOGL         JNJ  \
Date                                                                     
2018-01-02 00:00:00+00:00  40.568932  59.450500  53.527954  115.682426   
2018-01-03 00:00:00+00:00  40.561859  60.209999  54.441196  116.787521   
2018-01-04 00:00:00+00:00  40.750278  60.479500  54.652668  116.779213   
2018-01-05 00:00:00+00:00  41.214218  61.457001  55.377377  117.743019   
2018-01-08 00:00:00+00:00  41.061150  62.343498  55.572891  117.892570   

Ticker                           JPM       MSFT         PG        XOM  
Date                                                                   
2018-01-02 00:00:00+00:00  88.566315  79.792923  75.722343  61.637558  
2018-01-03 00:00:00+00:00  88.656609  80.164238  75.630470  62.848103  
2018-01-04 00:00:00+00:00  89.926651  80.869804  76.165085  62.935089  
2018-01-05 00:00:00+00:00  89.349342  81.872459  76.215210  62.884357  
2018-01-08 00:00:00+00:00  89.481300  81.956009  

Data Processing
Calculation of the daily returns and prepare the data for portfolio optimization.

In [14]:
# Calculate daily returns
returns = prices.pct_change().dropna()

# Display the first few rows of returns
print(returns.head())


Ticker                         AAPL      AMZN     GOOGL       JNJ       JPM  \
Date                                                                          
2018-01-03 00:00:00+00:00 -0.000174  0.012775  0.017061  0.009553  0.001020   
2018-01-04 00:00:00+00:00  0.004645  0.004476  0.003884 -0.000071  0.014325   
2018-01-05 00:00:00+00:00  0.011385  0.016163  0.013260  0.008253 -0.006420   
2018-01-08 00:00:00+00:00 -0.003714  0.014425  0.003531  0.001270  0.001477   
2018-01-09 00:00:00+00:00 -0.000115  0.004676 -0.001274  0.015857  0.005069   

Ticker                         MSFT        PG       XOM  
Date                                                     
2018-01-03 00:00:00+00:00  0.004653 -0.001213  0.019640  
2018-01-04 00:00:00+00:00  0.008802  0.007069  0.001384  
2018-01-05 00:00:00+00:00  0.012398  0.000658 -0.000806  
2018-01-08 00:00:00+00:00  0.001020  0.005261  0.004496  
2018-01-09 00:00:00+00:00 -0.000680 -0.007305 -0.004246  


Portfolio Construction
We'll construct two portfolios:

Equally Weighted Portfolio: Each asset has an equal weight.
Risk Parity Portfolio: Each asset contributes equally to the overall portfolio risk.


A. Equally Weighted Portfolio

In [15]:
# Number of assets
num_assets = len(tickers)

# Equal weights
equal_weights = pd.Series([1/num_assets]*num_assets, index=tickers)

print("Equally Weighted Portfolio:")
print(equal_weights)


Equally Weighted Portfolio:
AAPL     0.125
MSFT     0.125
AMZN     0.125
JNJ      0.125
JPM      0.125
XOM      0.125
PG       0.125
GOOGL    0.125
dtype: float64


B. Risk Parity Portfolio


In [16]:
import riskfolio as rp

# Create a Portfolio object
port = rp.Portfolio(returns=returns)

# Estimate the mean and covariance using historical data
port.assets_stats(method_mu='hist', method_cov='hist')

# Perform Risk Parity optimization
risk_parity_weights = port.optimization(model='RiskParity')

print("Risk Parity Portfolio Weights:")
print(risk_parity_weights)


ValueError: object arrays are not supported

Performance Analysis
Compare the performance metrics of both portfolios over the same period.

In [ ]:
# Function to calculate portfolio returns
def portfolio_return(weights, returns):
    return returns.dot(weights)

# Calculate daily returns for both portfolios
equal_portfolio_returns = portfolio_return(equal_weights, returns)
risk_parity_portfolio_returns = portfolio_return(risk_parity_weights, returns)

# Combine into a DataFrame
portfolio_returns = pd.DataFrame({
    'Equally Weighted': equal_portfolio_returns,
    'Risk Parity': risk_parity_portfolio_returns
})

# Calculate cumulative returns
cumulative_returns = (1 + portfolio_returns).cumprod()

# Display cumulative returns
print(cumulative_returns.tail())


Performance Metrics:

Calculate key performance metrics such as:

Cumulative Returns
Annualized Return
Annualized Volatility
Sharpe Ratio
Maximum Drawdown

In [ ]:
import numpy as np

def performance_metrics(returns):
    metrics = {}
    metrics['Cumulative Return'] = (1 + returns).prod() - 1
    metrics['Annualized Return'] = returns.mean() * 252
    metrics['Annualized Volatility'] = returns.std() * np.sqrt(252)
    metrics['Sharpe Ratio'] = metrics['Annualized Return'] / metrics['Annualized Volatility']
    metrics['Maximum Drawdown'] = (returns.cumprod().cummax() - returns.cumprod()).max()
    return metrics

# Calculate metrics for both portfolios
metrics_equal = performance_metrics(equal_portfolio_returns)
metrics_risk_parity = performance_metrics(risk_parity_portfolio_returns)

# Create a DataFrame for comparison
metrics_df = pd.DataFrame({
    'Equally Weighted': metrics_equal,
    'Risk Parity': metrics_risk_parity
})

print(metrics_df)


##Visualization
Visualize the portfolio allocations and performance.

A. Portfolio allocation pychart

In [ ]:
import matplotlib.pyplot as plt

# Plot Equally Weighted Portfolio
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
equal_weights.plot.pie(autopct='%.2f%%', startangle=90)
plt.title('Equally Weighted Portfolio Allocation')

# Plot Risk Parity Portfolio
plt.subplot(1, 2, 2)
risk_parity_weights.plot.pie(autopct='%.2f%%', startangle=90)
plt.title('Risk Parity Portfolio Allocation')

plt.tight_layout()
plt.show()


B. Cumulative Returns Comparison

In [ ]:
import seaborn as sns

# Plot cumulative returns
plt.figure(figsize=(10, 6))
sns.lineplot(data=cumulative_returns)
plt.title('Cumulative Returns Comparison')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend(loc='best')
plt.show()


C. Rolling Sharpe Ratio
Calculate and plot the rolling Sharpe ratio to assess the risk-adjusted performance over time.

In [ ]:
rolling_window = 252  # 1 year

# Calculate rolling Sharpe Ratio
rolling_sharpe_equal = (equal_portfolio_returns.rolling(window=rolling_window).mean() / 
                        equal_portfolio_returns.rolling(window=rolling_window).std()) * np.sqrt(252)

rolling_sharpe_risk = (risk_parity_portfolio_returns.rolling(window=rolling_window).mean() / 
                       risk_parity_portfolio_returns.rolling(window=rolling_window).std()) * np.sqrt(252)

# Combine into DataFrame
rolling_sharpe = pd.DataFrame({
    'Equally Weighted': rolling_sharpe_equal,
    'Risk Parity': rolling_sharpe_risk
})

# Plot rolling Sharpe Ratio
plt.figure(figsize=(10, 6))
sns.lineplot(data=rolling_sharpe)
plt.title('Rolling Sharpe Ratio (1-Year Window)')
plt.xlabel('Date')
plt.ylabel('Sharpe Ratio')
plt.legend(loc='best')
plt.show()
